# Tobig's 14기 2주차 Optimization 과제
### Made by 이지용

# Gradient Descent 구현하기

### 1) "..." 표시되어 있는 빈 칸을 채워주세요  
### 2) 강의내용과 코드에 대해 공부한 내용을 적어서 과제를 채워주세요

In [1]:
pip install -U scikit-learn

     |████████████████████████████████| 6.8MB 4.4MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [2]:
import pandas as pd
import numpy as np
import random
import tensorflow as tf
import math

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
data = pd.read_csv('/content/drive/My Drive/투빅스/2주차/assignment_2.csv')
data.head()

,Label,bias,experience,salary
0,1,1,0.7,48000
1,0,1,1.9,48000
2,1,1,2.5,60000
3,0,1,4.2,63000
4,0,1,6.0,76000


In [5]:
data.iloc[:, 1:]

,bias,experience,salary
0,1,0.7,48000
1,1,1.9,48000
2,1,2.5,60000
3,1,4.2,63000
4,1,6.0,76000
...,...,...,...
195,1,6.5,84000
196,1,6.9,73000
197,1,5.1,72000
198,1,9.1,69000


In [6]:
data.iloc[:, 0]

0      1
1      0
2      1
3      0
4      0
      ..
195    0
196    0
197    0
198    1
199    1
Name: Label, Length: 200, dtype: int64

## Train Test 데이터 나누기
### 데이터셋을 train/test로 나눠주는 메소드  
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0], test_size=0.25, random_state = 0)

In [9]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((150, 3), (50, 3), (150,), (50,))

## Scaling  

experience와 salary의 단위, 평균, 분산이 크게 차이나므로 scaler를 사용해 단위를 맞춰줍니다. 

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
bias_train = X_train["bias"]
bias_train = bias_train.reset_index()["bias"]
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
X_train["bias"] = bias_train
X_train.head()

,bias,experience,salary
0,1,0.187893,-1.143335
1,1,1.185555,0.043974
2,1,-0.310938,-0.351795
3,1,-1.629277,-1.341220
4,1,-1.308600,0.043974


이때 scaler는 X_train에 fit 해주시고, fit한 scaler를 X_test에 적용시켜줍니다.  
똑같이 X_test에다 fit하면 안돼요!


In [11]:
bias_test = X_test["bias"]
bias_test = bias_test.reset_index()["bias"]
X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)
X_test["bias"] = bias_test
X_test.head()

,bias,experience,salary
0,1,-1.344231,-0.615642
1,1,0.508570,0.307821
2,1,-0.310938,0.571667
3,1,1.363709,1.956862
4,1,-0.987923,-0.747565


In [12]:
# parameter 개수
N = len(X_train.loc[0])

In [13]:
# 초기 parameter들을 임의로 설정해줍니다.
parameters = np.array([random.random() for i in range(N)])
parameters

array([0.50158737, 0.96837671, 0.12686596])

### * LaTeX   

Jupyter Notebook은 LaTeX 문법으로 수식 입력을 지원하고 있습니다.  
http://triki.net/apps/3466  
https://jjycjnmath.tistory.com/117

## Logistic Function

## $p = "\frac{1}{1 + e^{-(\hat{\beta_0}+\hat{\beta_1}x_1+...+\hat{\beta_k}x_k)}}"$



In [14]:
def logistic(X, parameters):
    z = 0
    for i in range(len(parameters)):
      z += parameters[i]*X[i]
    p = 1/(1+np.exp(-z))
    
    return p

In [15]:
logistic(X_train.iloc[1], parameters)

0.8395966851814499

## Object Function

Object Function : 목적함수는 Gradient Descent를 통해 최적화 하고자 하는 함수입니다.  
로지스틱 회귀의 목적함수를 작성해주세요
## $l(p) ="-\sum[{(y_i)logp+(1-y_i)log(1-p)}]"$




In [16]:
def minus_log_cross_entropy_i(X, y, parameters) :
  p = logistic(X, parameters)   
  loss = -np.sum(y*np.log(p)+(1-y)*np.log(1-p))
  return loss

In [17]:
def minus_log_cross_entropy(X_set, y_set, parameters) :
    loss = 0
    for i in range(X_set.shape[0]):
        X = X_set.iloc[i, :]
        y = y_set.iloc[i]
        loss += minus_log_cross_entropy_i(X,y,parameters)
    return loss

In [18]:
minus_log_cross_entropy(X_test, y_test, parameters)

54.22860911837079

## Gradient of Minus Log Cross Entropy

## ${\partial\over{\partial \theta_j}}l(p)= "-\sum[(y_i-p_i)x_{ij}]"$




In [109]:
# cross_entropy를 theta_j에 대해 미분한 값을 구하는 함수
def get_gradient_ij_minus_log_cross_entropy(X, y, parameters, j):
    p = logistic(X, parameters)
    gradient = -((y-p)*X[j])
    return gradient

In [110]:
get_gradient_ij_minus_log_cross_entropy(X_train.iloc[0, :], y_train.iloc[0], parameters, 1)

-0.06851691462927095

## Batch Gradient Descent  

Batch Gradient Descent : 
$w=w-\eta \nabla Q(w)=w-\eta \sum_{i=1}^{n} \nabla Q_{i}(w)$






In [105]:
def get_gradients_bgd(X_train, y_train, parameters) :
    gradients = [0 for i in range(len(parameters))]
    
    for i in range(X_train.shape[0]):
        X = X_train.iloc[i, :]
        y = y_train.iloc[i]
        for j in range(len(parameters)):
            gradients[j] += get_gradient_ij_minus_log_cross_entropy(X, y, parameters, j)
            
    return gradients

In [106]:
gradients_bgd = get_gradients_bgd(X_train, y_train, parameters)
gradients_bgd

[47.68336369643797, 14.23812534021317, 39.42992896053932]

## Stochastic Gradient Descent  

Stochastic Gradient Descent : $Q(w)=\sum_{i=1}^nQi(w)$

In [74]:
def get_gradients_sgd(X_train, y, parameters) :
    gradients = [0 for i in range(len(parameters))]
    r = int(random.random()*X_train.shape[0])
    X = X_train.iloc[r, :]
    y = y_train.iloc[r]
        
    for j in range(len(parameters)):
        gradients[j] = get_gradient_ij_minus_log_cross_entropy(X, y, parameters, j)
        
    return gradients

In [75]:
gradients_sgd = get_gradients_sgd(X_train, y_train, parameters)
gradients_sgd

[0.31863503499724916, -0.41696587205189395, 0.014011791289616377]

## Update Parameters  

In [76]:
def update_parameters(parameters, gradients, learning_rate) :
    for i in range(len(parameters)) :
        gradients[i] *= learning_rate
    parameters -= gradients
    return parameters

In [77]:
update_parameters(parameters, gradients_bgd, 0.01)

array([0.51148796, 0.96953705, 0.12110067])

## Gradient Descent  

위에서 작성한 함수들을 조합해서 Gradient Descent를 진행하는 함수를 완성해주세요

learning_rate = 0.01  
max_iter = 100000  
tolerance = 0.0001


In [111]:
def gradient_descent(X_train, y_train, learning_rate=0.01, max_iter=100000, tolerance=0.0001, optimizer="bgd") :
    count = 1
    point = 100 if optimizer == "bgd" else 10000
    N = len(X_train.iloc[0])
    parameters = np.array([random.random() for i in range(N)])
    gradients = [0 for i in range(N)]
    loss = 0
    
    while count < max_iter :
        
        if optimizer == "bgd" :
            gradients =  get_gradients_bgd(X_train, y_train, parameters)
        elif optimizer == "sgd" :
            gradients = get_gradients_sgd(X_train, y_train, parameters)
            # loss, 중단 확인
        if count%point == 0 :
            new_loss = minus_log_cross_entropy(X_train, y_train, parameters)
            print(count, "loss: ",new_loss, "params: ", parameters, "gradients: ", gradients)
            
            #중단 조건
            if abs(new_loss-loss) < tolerance*len(y_train) :
                break
            loss = new_loss
                
            
                
        parameters = update_parameters(parameters, gradients, learning_rate)
        count += 1
    return parameters

In [112]:
new_param_bgd = gradient_descent(X_train, y_train)
new_param_bgd

100 loss:  45.40007360678799 params:  [-1.62116297  3.46541824 -3.29230436] gradients:  [0.280929333869887, -0.9247006325856403, 0.8645213956100467]
200 loss:  44.801902602054206 params:  [-1.78112395  3.98835037 -3.77856763] gradients:  [0.08427349336006698, -0.2738640803850837, 0.25328766964719873]
300 loss:  44.74043314253642 params:  [-1.83377984  4.15919713 -3.93631575] gradients:  [0.030311434769616355, -0.09819595421104044, 0.09051498986181339]
400 loss:  44.73215307400144 params:  [-1.85324704  4.22222891 -3.99438237] gradients:  [0.011517947281423205, -0.03727331792377904, 0.03431629847949706]


array([-1.85324704,  4.22222891, -3.99438237])

## Hyper Parameter Tuning

Hyper Parameter들을 매번 다르게 해서 학습을 진행해 보세요. 다른 점들을 발견할 수 있습니다.

In [113]:
new_param_sgd = gradient_descent(X_train, y_train, learning_rate=0.01, max_iter=100000, tolerance=0.0001, optimizer="sgd")
new_param_sgd

10000 loss:  46.41017148105004 params:  [-1.55727262  3.05045612 -2.91215816] gradients:  [0.009825949911665521, -0.002004942497239956, 0.008209718772223982]
20000 loss:  45.21771001934296 params:  [-1.68078212  3.60622805 -3.56128384] gradients:  [0.002866553996234679, -0.003751173146525313, -0.0004411926865042025]
30000 loss:  44.82084034868825 params:  [-1.81382946  4.03626868 -3.76311841] gradients:  [0.17880941737158368, -0.14479434507404243, -0.16905537425374068]
40000 loss:  44.891483673557566 params:  [-1.71457195  4.2349263  -3.93536189] gradients:  [0.018961782555588447, -0.025489015857395353, -0.016676668317423957]
50000 loss:  44.743399816758206 params:  [-1.87531533  4.18171464 -3.96995204] gradients:  [0.07997117932678735, -0.0676076281635997, -0.059783667720841836]
60000 loss:  44.75975155146625 params:  [-1.79384951  4.20553109 -3.99990019] gradients:  [-0.3758108629994159, 0.3444913484928286, 0.5784122834663703]
70000 loss:  44.853305644155064 params:  [-1.91626303  4.

array([-1.82924768,  4.38866211, -4.02883547])

## Predict Label

In [114]:
y_predict = []
for i in range(len(y_test)):
    p = logistic(X_test.iloc[i,:], new_param_bgd)
    if p> 0.5 :
        y_predict.append(1)
    else :
        y_predict.append(0)

## Confusion Matrix

In [115]:
from sklearn.metrics import *
tn, fp, fn, tp = confusion_matrix(y_test, y_predict).ravel()
confusion_matrix(y_test, y_predict)

array([[38,  2],
       [ 1,  9]])